In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch_geometric.utils import to_dense_adj

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # print(adjacency.shape, x.shape)
        # print(x)
        x = torch.sparse.mm(adjacency, x)
        return x

In [4]:
dataset = CVFConfigDataset()
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

# # x = torch.tensor([[0], [0], [0], [0]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]
# x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

# y = torch.tensor([[3.0, 1.0]]) # ranks

# data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)
# data.train_mask = torch.tensor([1 for _ in range(len(x))])

In [6]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency

tensor([[1., 1., 1., 1.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.],
        [1., 0., 0., 1.]])

In [7]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)
        # self.out = torch.nn.Linear(dim_h, dim_out)
    
    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        # h = torch.relu(h)
        # print("h2 r", h)
        # h = self.out(h)
        # print(torch.relu(h))
        result = F.softmax(h, dim=0)
        # print("result", result)
        return result
    
    def fit(self, data_loader, epochs):
        # criterion = torch.nn.MSELoss()
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001, weight_decay=5e-4)
        self.train()
        for epoch in range(epochs+1):
            for batch in data_loader:
                x = batch[0][0]
                y = torch.zeros(4, 1)
                y[batch[1]] = 1.0
                # y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency)
                out = out.flatten()
                y = y.flatten()
                # print("output", out, "y", y)
                loss = criterion(out, y)
                print("loss", loss)
                loss.backward()
                optimizer.step()
            
            # if loss < 0.009:
            #     print("Loss threshold met.")
            #     break
            print("Loss:", loss)


In [8]:
gnn = VanillaGNN(1, 64, 1)
print(gnn)
gnn.fit(data_loader, epochs=100)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=1, out_features=64, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=64, out_features=1, bias=False)
  )
)
loss tensor(1.4079, grad_fn=<DivBackward1>)
loss tensor(1.4119, grad_fn=<DivBackward1>)
loss tensor(1.3942, grad_fn=<DivBackward1>)
loss tensor(1.4054, grad_fn=<DivBackward1>)
loss tensor(1.3955, grad_fn=<DivBackward1>)
loss tensor(1.3890, grad_fn=<DivBackward1>)
loss tensor(1.3856, grad_fn=<DivBackward1>)
loss tensor(1.3805, grad_fn=<DivBackward1>)
loss tensor(1.3760, grad_fn=<DivBackward1>)
loss tensor(1.3666, grad_fn=<DivBackward1>)
loss tensor(1.3609, grad_fn=<DivBackward1>)
loss tensor(1.3662, grad_fn=<DivBackward1>)
loss tensor(1.3732, grad_fn=<DivBackward1>)
loss tensor(1.3338, grad_fn=<DivBackward1>)
loss tensor(1.3596, grad_fn=<DivBackward1>)
loss tensor(1.3436, grad_fn=<DivBackward1>)
loss tensor(1.3409, grad_fn=<DivBackward1>)
loss tensor(1.3219, grad_fn=<DivBackward1>)
lo

In [9]:
# out = gnn(torch.tensor([[3.], [2.], [2.], [2.]]), adjacency)
# print(out)

In [10]:
for batch in data_loader:
    x = batch[0][0]
    y = batch[1]
    predicted = gnn(x, adjacency).argmax(dim=0)
    print("y", y, "predicted", predicted)

y tensor([3]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([3]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([3]) predicted tensor([0])
y tensor([0]) predicted tensor([0])
y tensor([0]) predicted tensor([0])
y tensor([1]) predicted tensor([0])
y tensor([2]) predicted tensor([0])
y tensor([3]) predicted tensor([0])
y tensor([0]) predicted tensor([0])
y tensor([0]) predicted tensor([0])
y tensor([0]) predicted tens